In [1]:
from plotly import graph_objs as go
from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx
import dash
import dash_core_components as dcc
import dash_html_components as html
import networkx as nx
import plotly.graph_objs as go

import pandas as pd
from colour import Color
from datetime import datetime
from textwrap import dedent as d
import json

In [2]:
finance = pd.read_csv('finance.csv',index_col='Unnamed: 0').dropna()
df_tags =finance[:100]
df_test = pd.DataFrame(cosine_similarity(df_tags.T),columns=df_tags.T.index.tolist(),index=df_tags.T.index.tolist())

In [3]:
li = []
[li.extend([(i,j) for j in df_test.ix[i].items()]) for i in df_test.index]

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



[None, None, None, None, None, None, None, None, None, None, None, None]

In [4]:
dict_head={
    'tag':[i[0] for i in li],
    'tag2':[i[1][0] for i in li],
    'distance':[("%.2f" % i[1][1])for i in li]
}
df_distance = pd.DataFrame(dict_head)

In [5]:
#每個人都與自己聯繫在一起
df_distance = df_distance[~(df_distance['tag']==df_distance['tag2'])].dropna()[['tag', 'tag2', 'distance']]

In [6]:
#當X和Y相連時，Y也和X相連，同一個連接有兩行
df_distance.drop(df_distance.loc[df_distance['tag2'] < df_distance['tag']].index.tolist(), inplace=True)

In [7]:
df_test['tag']=[i for i in df_test.index.tolist() ]

In [8]:
#將節點連線
G = nx.from_pandas_edgelist(df_distance, source='tag', target='tag2', edge_attr='distance')

In [9]:
nx.set_node_attributes(G, {row['tag']:{'Name': row['tag']} for i,row in df_test.iterrows()})

In [10]:
pos = nx.layout.spring_layout(G)
for node in G.nodes:
    G.nodes[node]['pos'] = list(pos[node])

In [11]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

In [12]:
node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

In [51]:
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(adjacencies[0])

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [52]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
#                 titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )


In [53]:
fig.show()